# Payment Behaviour

1. payment_method ↔ invoice_date <br>
What are the trends in payment method shares over time?

2. payment_method ↔ gender <br>
How does payment method preference vary by gender?

3. payment_method ↔ Econ_Conf_level <br>
Does card usage increase when consumer confidence declines?

4. payment_method ↔ category <br>
Does payment method choice differ by product category?

5. payment_method ↔ basket_total_price <br>
Which payment methods are used for high-value baskets?

6. credit_card_share ↔ Econ_Conf (lag 0/+1) <br>
How does the credit card usage rate respond to changes in the consumer confidence index?